In [1]:
using Revise, Pkg
Pkg.activate("/home/louise/MSA/BpAlignGpu.jl")
using BpAlignGpu

  Activating project at `~/MSA/BpAlignGpu.jl`
┌ Info: Precompiling BpAlignGpu [5a3eb610-29b2-4cbe-8ba2-ea97f65fa95d]
└ @ Base loading.jl:1423


In [ ]:
using CUDA
CUDA.device!(1)

In [2]:
#(q, N, L) = (21, 161, 67) 
(q, N, L) = (21, 5, 5) 

(21, 5, 5)

In [3]:
function convert_T3_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, L)
    for i=1:L
        for n=1:N+2
            SCE_ar[n, i] = tmp[n,1,i]
            SCE_ar[N+2+n, i] = tmp[n,2,i]
        end
    end
    return SCE_ar
end
function convert_T5_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L)
    for i=1:L
        for n1=1:N+2
            for n2=1:N+2
                SCE_ar[n1, n2, i] = tmp[n1, 1, n2, 1, i]
                SCE_ar[n1, N+2+n2, i] = tmp[n1, 1, n2, 2, i]
                SCE_ar[N+2+n1, n2, i] = tmp[n1, 2, n2, 1, i]
                SCE_ar[N+2+n1, N+2+n2, i] = tmp[n1, 2, n2, 2, i]
            end
        end
    end
    return SCE_ar
end
function convert_T6_xnTOy(tmp)
    SCE_ar = fill(0.0, 2N+4, 2N+4, L, L)
    for i=1:L
        for j=1:L
            for n1=1:N+2
                for n2=1:N+2
                    SCE_ar[n1, n2, i, j] = tmp[n1, 1, n2, 1, i, j]
                    SCE_ar[n1, N+2+n2, i, j] = tmp[n1, 1, n2, 2, i, j]
                    SCE_ar[N+2+n1, n2, i, j] = tmp[n1, 2, n2, 1, i, j]
                    SCE_ar[N+2+n1, N+2+n2, i, j] = tmp[n1, 2, n2, 2, i, j]
                end
            end
        end
    end
    return SCE_ar
end

convert_T6_xnTOy (generic function with 1 method)

In [4]:
T = Float32

Float32

In [5]:
using Random
header = "myseq"
myseq = randstring('A':'Z', N)
ctype=Symbol("amino")
seq = Seq(header, myseq, ctype)

myseq
EMBGC

In [6]:
muint = 1.5
muext = 1.2
lambda_o = ones(L)
lambda_e = ones(L)
H = rand(q,L)
J = rand(q,q,L,L)
J = J .+ permutedims(J, (2,1,4,3));

In [7]:
pm = ParamModel{T}(N, L, q, muint, muext, lambda_o, lambda_e, H, J)

ParamModel{Float32}[L=5 N=5 q=21 size=43.547 KiB]

In [8]:
damp=T(0.0)
tol=T(1e-5)
tolnorm=T(1e-5)
tmax=10
upscheme=:sequential # :random or :sequential
lr=:sce  # :sce or :mf 
beta=T(1.0)
verbose=true
pa = ParamAlgo(damp, tol, tolnorm, tmax, upscheme, lr, beta, verbose)

ParamAlgo{Float32}
-------------
damp=0.0
tol=1.0e-5
tolnorm=1.0e-5
tmax=10
upscheme=sequential
lr=sce
beta=1.0
verbose=true
-------------

In [9]:
bpm = BPMessages(seq, pm)

┌ Warning: The NVIDIA driver on this system only supports up to CUDA 11.0.0.
│ For performance reasons, it is recommended to upgrade to a driver that supports CUDA 11.2 or higher.
└ @ CUDA /home/louise/.julia/packages/CUDA/bki2w/src/initialization.jl:70


BPMessages{Float32}[L=5 N=5 ongpu=true size=20.820 KiB]

In [10]:
bpb = BPBeliefs(N, L)

BPBeliefs{Float32}[L=5 N=5 ongpu=true size=23.516 KiB]

In [11]:
lrf = LongRangeFields(N, L)

LongRangeFields{Float32}[L=5 N=5 ongpu=true size=80.664 KiB]

In [12]:
af = AllFields(bpm, bpb, lrf)

AllFields{Float32}[L=5 N=5 ongpu=true size=125.000 KiB]

# Checks

In [15]:
tmp_Jseq = Array(af.bpm.Jseq)
elts_Jseq = convert_T6_xnTOy(tmp_Jseq);

In [16]:
tmp_Hseq = Array(af.bpm.Hseq)
elts_Hseq = convert_T3_xnTOy(tmp_Hseq);

In [90]:
include("../src/SCE_update_messages.jl")
include("../src/SCE_update_shortrange_beliefs.jl")
include("../src/SCE_update_longrange.jl")

update_lr_free_nrj (generic function with 1 method)

# check BP updates

In [18]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);

tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

SCE_hF = zeros(2*(N+2),L);

In [19]:
BpAlignGpu.update_hF!(af, pm, pa)
#BpAlignGpu.update_hB!(af, pm, pa)
#BpAlignGpu.update_F!(af, pm, pa)
#BpAlignGpu.update_B!(af, pm, pa)

In [20]:
i=2
af.bpm.hF[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0266276  0.0
 0.0285996  0.0535879
 0.0990783  0.0585607
 0.0812063  0.136227
 0.0198478  0.135813
 0.0319143  0.164934
 0.163603   0.0

In [21]:
#SCE_update_F!(SCE_F, i, damp, beta, tolnorm, SCE_hF, SCE_f, muext, muint, N, elts_Hseq)
#SCE_update_B!(SCE_B, i, damp, beta, tolnorm, SCE_hB, SCE_f, muext, muint, N, L, elts_Hseq)
SCE_update_hF!(SCE_hF, i, SCE_F, damp, beta, tolnorm, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)
#SCE_update_hB!(SCE_hB, i, damp, beta, tolnorm, SCE_B, SCE_g, pm.lambda_o, pm.lambda_e, N, elts_Jseq)

In [22]:
SCE_hF[:,i]

14-element Vector{Float64}:
 0.02662764186273695
 0.028599615907551917
 0.09907828252446754
 0.08120628048813693
 0.019847832622820746
 0.031914280843717224
 0.16360339391701023
 0.0
 0.053587940008133
 0.05856065715297686
 0.13622713510499257
 0.13581289919215597
 0.16493404037530013
 0.0

# check update beliefs

In [23]:
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

SCE_bel = zeros(2*(N+2),L);

In [24]:
BpAlignGpu.update_beliefs!(af, pm, pa)

In [28]:
i=5
SCE_update_beliefs!(SCE_bel, i, beta, tolnorm, SCE_hF, SCE_hB, SCE_f, muint, muext, N, L, elts_Hseq)

-0.1491311285436651

In [29]:
af.bpb.beliefs[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0        0.0
 0.0        0.0370461
 0.0        0.21977
 0.0        0.281647
 0.0        0.280348
 0.0        0.0820077
 0.0991807  0.0

In [30]:
SCE_bel[:,i]

14-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.09918070824327868
 0.0
 0.03704613389394673
 0.21976992541922738
 0.2816471860575254
 0.28034839815405915
 0.08200764823196258
 0.0

In [31]:
logzi, mynorm_Zi = BpAlignGpu.logZi(af, pm, pa)

(-9.228255f0, [-0.568089;;; -2.821702;;; -2.412248;;; -3.2770858;;; -0.14913118])

In [32]:
mynorm_Zi[i]

┌ Warning: Performing scalar indexing on task Task (runnable) @0x00007f3341391b60.
│ Invocation of getindex resulted in scalar indexing of a GPU array.
│ This is typically caused by calling an iterating implementation of a method.
│ Such implementations *do not* execute on the GPU, but very slowly on the CPU,
│ and therefore are only permitted from the REPL for prototyping purposes.
│ If you did intend to index this array, annotate the caller with @allowscalar.
└ @ GPUArrays /home/louise/.julia/packages/GPUArrays/umZob/src/host/indexing.jl:56


-0.14913118f0

# check update joint proba

In [33]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);

tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

SCE_joint = zeros(2(N+2), 2(N+2), L);

In [35]:
BpAlignGpu.update_jointchain!(af, pm, pa)

In [47]:
i=4
af.bpb.joint_chain[:,1,:,1,i]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.00260969  0.0        0.0      0.0     0.0         0.0        0.0
 0.0         0.0858345  0.0      0.0     0.0         0.0        0.0
 0.0         0.0        0.04838  0.0     0.0         0.0        0.0
 0.0         0.0        0.0      0.0702  0.0         0.0        0.0
 0.0         0.0        0.0      0.0     0.00122838  0.0        0.0
 0.0         0.0        0.0      0.0     0.0         0.0167884  0.0
 0.0         0.0        0.0      0.0     0.0         0.0        0.0287026

In [48]:
SCE_update_joint_chain!(SCE_joint, i, beta, tolnorm, SCE_F, SCE_B, SCE_g, lambda_o, lambda_e, N, elts_Jseq)

-0.07041188161442055

In [49]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_joint[rx1,rx1, i]

7×7 Matrix{Float64}:
 0.00260969  0.0        0.0      0.0     0.0         0.0        0.0
 0.0         0.0858345  0.0      0.0     0.0         0.0        0.0
 0.0         0.0        0.04838  0.0     0.0         0.0        0.0
 0.0         0.0        0.0      0.0702  0.0         0.0        0.0
 0.0         0.0        0.0      0.0     0.00122838  0.0        0.0
 0.0         0.0        0.0      0.0     0.0         0.0167884  0.0
 0.0         0.0        0.0      0.0     0.0         0.0        0.0287026

In [50]:
logza, mynormza = BpAlignGpu.logZa(af, pm, pa)

(-1.2205039f0, [-0.5746991;;;;; -0.10622883;;;;; -0.46916407;;;;; -0.07041193;;;;; 0.0])

In [51]:
mynormza[i]

-0.07041193f0

# Check logZedge

In [52]:
BpAlignGpu.logZia(af, pm)

-19.893305f0

In [61]:
tmp_F = Array(af.bpm.F)
SCE_F = convert_T3_xnTOy(tmp_F);
tmp_B = Array(af.bpm.B)
SCE_B = convert_T3_xnTOy(tmp_B);
tmp_hF = Array(af.bpm.hF)
SCE_hF = convert_T3_xnTOy(tmp_hF);
tmp_hB = Array(af.bpm.hB)
SCE_hB = convert_T3_xnTOy(tmp_hB);

In [73]:
logZedge(SCE_F, SCE_B, SCE_hF, SCE_hB)

(-9.793128815790165, -10.100176622177495, -19.893305437967662)

# check update conditional on the chain

In [74]:
BpAlignGpu.update_conditional_chain!(af, pa)

In [75]:
tmp_joint = Array(af.bpb.joint_chain)
SCE_joint = convert_T5_xnTOy(tmp_joint);

In [76]:
cond_chain = fill(0.0, 2N+4, 2N+4, L, L)
for i=1:L-1
    SCE_update_conditional_chain!(i, cond_chain, SCE_joint, tolnorm, N)
end

In [77]:
af.bpb.conditional[:,2,:,1,1,2]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0        0.0       0.0       0.0       0.0      0.0
 0.0  0.0750415  0.0       0.0       0.0       0.0      0.0131181
 0.0  0.0        0.551304  0.0       0.0       0.0      0.33387
 0.0  0.0        0.0       0.485617  0.0       0.0      0.241041
 0.0  0.0        0.0       0.0       0.849384  0.0      0.103045
 0.0  0.0        0.0       0.0       0.0       0.58122  0.113379
 0.0  0.0        0.0       0.0       0.0       0.0      0.0

In [78]:
cond_chain[rx2, rx1, 1, 2]

7×7 Matrix{Float64}:
 0.0  0.0        0.0       0.0       0.0       0.0      0.0
 0.0  0.0750415  0.0       0.0       0.0       0.0      0.0131181
 0.0  0.0        0.551304  0.0       0.0       0.0      0.33387
 0.0  0.0        0.0       0.485617  0.0       0.0      0.241041
 0.0  0.0        0.0       0.0       0.849384  0.0      0.103045
 0.0  0.0        0.0       0.0       0.0       0.58122  0.113379
 0.0  0.0        0.0       0.0       0.0       0.0      0.0

# check update conditional off the chain

In [81]:
BpAlignGpu.update_conditional_all!(af, pm);

In [83]:
cond_lr = copy(cond_chain);
pointer(cond_chain), pointer(cond_lr)

(Ptr{Float64} @0x000000000e870380, Ptr{Float64} @0x000000003545edc0)

In [84]:
SCE_update_conditional_lr!(cond_lr)

In [85]:
cond_lr[rx1, rx2, 4, 1]

7×7 Matrix{Float64}:
 0.0  0.0         0.0        0.0         0.0        0.0       0.0
 0.0  0.0999931   0.0        0.0         0.0        0.0       0.0
 0.0  0.0610358   0.0163893  0.0         0.0        0.0       0.0
 0.0  0.0694525   0.0264041  0.00504247  0.0        0.0       0.0
 0.0  0.00660926  0.0159084  0.040201    0.0579838  0.0       0.0
 0.0  0.0290143   0.0693638  0.227492    0.359661   0.602773  0.0
 0.0  0.482484    0.777205   0.610753    0.469369   0.397227  0.0

In [86]:
af.bpb.conditional[:,1,:,2,4,1]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 0.0  0.0         0.0        0.0         0.0        0.0       0.0
 0.0  0.0999931   0.0        0.0         0.0        0.0       0.0
 0.0  0.0610358   0.0163893  0.0         0.0        0.0       0.0
 0.0  0.0694525   0.0264041  0.00504247  0.0        0.0       0.0
 0.0  0.00660926  0.0159084  0.040201    0.0579838  0.0       0.0
 0.0  0.0290143   0.0693638  0.227492    0.359661   0.602773  0.0
 0.0  0.482484    0.777205   0.610753    0.469369   0.397227  0.0

# Check long range Free Energy

In [87]:
BpAlignGpu.lr_freeen(af, pm)

5.736133873462677

In [88]:
tmp_bel = Array(af.bpb.beliefs)
SCE_bel = convert_T3_xnTOy(tmp_bel)
tmp_cond = Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [92]:
using LinearAlgebra
update_lr_free_nrj(SCE_bel, SCE_cond, elts_Jseq)

5.73613381561961

# check update f

In [93]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [94]:
tmp_f = Array(af.lrf.f)
SCE_f = convert_T3_xnTOy(tmp_f);

In [95]:
BpAlignGpu.update_f!(af)

In [96]:
using LinearAlgebra
SCE_update_f!(SCE_f, SCE_cond, elts_Jseq, N)

In [97]:
i=3
af.lrf.f[:,:,i]

7×2 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 -3.55044   0.0
 -3.78626  -3.31763
 -3.45854  -3.31743
 -3.38101  -3.41131
 -3.72167  -3.23595
 -3.93591  -3.5331
 -3.78966   0.0

In [98]:
SCE_f[:,i]

14-element Vector{Float64}:
 -3.5504422532556457
 -3.786260287535633
 -3.458540082720221
 -3.381010354092398
 -3.7216719429517506
 -3.9359089373564355
 -3.789658690239371
  0.0
 -3.3176288431712444
 -3.3174334455535814
 -3.4113076768535056
 -3.2359514797722126
 -3.533102419624318
  0.0

# check update g

In [99]:
tmp_cond= Array(af.bpb.conditional)
SCE_cond = convert_T6_xnTOy(tmp_cond);

In [100]:
tmp_g = Array(af.lrf.g)
SCE_g = convert_T5_xnTOy(tmp_g);

In [101]:
BpAlignGpu.update_g!(af)

In [102]:
SCE_update_g!(SCE_g, SCE_cond, elts_Jseq, N)

In [107]:
i=3
af.lrf.g[:,1,:,1,i]

7×7 CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}:
 4.32139  4.45074  4.24295  4.20206  3.86758  4.25646  4.25646
 4.69293  4.89477  4.71678  4.64583  4.09708  4.74022  4.74022
 4.58479  4.61236  4.38959  4.38949  4.37733  4.38962  4.38962
 4.40647  4.51177  4.2974   4.26535  4.00175  4.30799  4.30799
 5.00463  5.06935  4.947    4.92831  4.80552  4.95317  4.95317
 5.29219  5.21567  4.99558  5.03758  5.38104  4.9817   4.9817
 4.9833   5.02899  4.83297  4.82545  4.77711  4.83545  4.83545

In [108]:
rx1 = 1:N+2
rx2 = N+3:2N+4
SCE_g[rx1,rx1,i]

7×7 Matrix{Float64}:
 4.32139  4.45074  4.24295  4.20206  3.86758  4.25646  4.25646
 4.69293  4.89477  4.71678  4.64583  4.09708  4.74022  4.74022
 4.58479  4.61236  4.38959  4.38949  4.37733  4.38962  4.38962
 4.40647  4.51177  4.2974   4.26535  4.00175  4.30799  4.30799
 5.00464  5.06935  4.947    4.92831  4.80552  4.95317  4.95317
 5.29219  5.21567  4.99558  5.03758  5.38104  4.9817   4.9817
 4.9833   5.02899  4.83297  4.82545  4.77711  4.83545  4.83545